"""
Created on Sat Jun 29 2025
Modified on Sat Jun 29 2025

@author: leowp2011
https://github.com/leowp2011/salesintelligence

Rotina:
    - Carrega a base de vendas internas (clientes reais da empresa);
    - Conecta à base pública da Receita Federal em SQLite (`cnpj.db`);
    - Identifica empresas com CNAEs compatíveis (primários e/ou secundários);
    - Gera a tabela fato `f_empresas_potenciais` com os seguintes indicadores:
        • cliente_atual (se já comprou)
        • possui_cnae_alvo_primario (match CNAE primário)
        • possui_cnae_alvo_secundario (match CNAE secundário)
        • possui_lead (caso haja leads externos disponíveis)
    - Enriquecer com:
        • Seção CNAE (letra + descrição)
        • Município
    - Salva as tabelas finais (fato e dimensões) em um novo banco SQLite (`gold_base_powerbi.db`) pronto para consumo no Power BI.

Observações:
    - Requer que a base `cnpj.db` esteja previamente gerada (veja https://github.com/rictom/cnpj-sqlite)
    - Arquivo de vendas deve estar disponível na pasta raiz, pois é com base neste arquivo que o estudo é feito (`../vendas.csv`)

Dependências:
    - pandas
    - sqlite3
"""


In [9]:
import pandas as pd, sqlite3, sqlalchemy

In [10]:
vendas = pd.read_csv("../inputs/vendas.csv", dtype={"cnpj": str})
clientes_cnpjs = vendas["cnpj"].dropna().unique().tolist()

In [11]:
conn = sqlite3.connect("../1 - Extract & Load/dados-publicos/cnpj.db")

In [12]:
# CNAEs primários dos clientes
cnaes_primarios = pd.read_sql(
    f"SELECT DISTINCT cnae_fiscal FROM estabelecimento WHERE cnpj IN ({','.join(['?'] * len(clientes_cnpjs))})",
    conn, params=clientes_cnpjs)["cnae_fiscal"].dropna().unique().tolist()

# CNAEs secundários dos clientes
cnaes_secundarios = pd.read_sql(
    f"SELECT DISTINCT cnae_fiscal_secundaria FROM cnae_secundaria WHERE cnpj IN ({','.join(['?'] * len(clientes_cnpjs))})",
    conn, params=clientes_cnpjs)["cnae_fiscal_secundaria"].dropna().unique().tolist()

cnaes_alvo = list(set(cnaes_primarios + cnaes_secundarios))

In [13]:
# CNAE primário
empresas_primario = pd.read_sql(
    f"""
    SELECT cnpj, municipio, cnae_fiscal, cnpj_basico
    FROM estabelecimento 
    WHERE cnae_fiscal IN ({','.join(['?'] * len(cnaes_alvo))})
      AND situacao_cadastral = '02'
    """,
    conn, params=cnaes_alvo)

# CNAE secundário
empresas_secundario = pd.read_sql(
    f"SELECT DISTINCT cnpj FROM cnae_secundaria WHERE cnae_fiscal_secundaria IN ({','.join(['?'] * len(cnaes_alvo))})",
    conn, params=cnaes_alvo)


In [14]:
todos_cnpjs = list(set(empresas_primario["cnpj"].tolist() + empresas_secundario["cnpj"].tolist()))
fato = pd.DataFrame({"cnpj": todos_cnpjs})

# Enriquecer com dados da tabela de empresas primárias
fato = fato.merge(empresas_primario[["cnpj", "municipio", "cnae_fiscal", "cnpj_basico"]], on="cnpj", how="left")

# Renomear para manter consistência com dimensões
fato.rename(columns={
    "municipio": "codigo_municipio",
    "cnae_fiscal": "codigo_cnae",
    "cnpj_basico": "raiz_cnpj"
}, inplace=True)

fato["cliente_saur"] = fato["cnpj"].isin(clientes_cnpjs).astype(int)
fato["possui_cnae_alvo_primario"] = fato["cnpj"].isin(empresas_primario["cnpj"]).astype(int)
fato["possui_cnae_alvo_secundario"] = fato["cnpj"].isin(empresas_secundario["cnpj"]).astype(int)
fato["possui_lead"] = 0  # você pode popular isso mais tarde


In [15]:
municipio = pd.read_sql("SELECT * FROM municipio", conn)
cnae = pd.read_sql("SELECT codigo, descricao FROM cnae", conn)

# Extrair divisão a partir dos 2 primeiros dígitos do código
cnae["divisao"] = cnae["codigo"].astype(str).str[:2].astype(int)

# Função de mapeamento para seção
def mapear_secao(div):
    if 1 <= div <= 3:
        return 'A', 'AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA'
    elif 5 <= div <= 9:
        return 'B', 'INDÚSTRIAS EXTRATIVAS'
    elif 10 <= div <= 33:
        return 'C', 'INDÚSTRIAS DE TRANSFORMAÇÃO'
    elif div == 35:
        return 'D', 'ELETRICIDADE E GÁS'
    elif 36 <= div <= 39:
        return 'E', 'ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS E DESCONTAMINAÇÃO'
    elif 41 <= div <= 43:
        return 'F', 'CONSTRUÇÃO'
    elif 45 <= div <= 47:
        return 'G', 'COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS'
    elif 49 <= div <= 53:
        return 'H', 'TRANSPORTE, ARMAZENAGEM E CORREIO'
    elif 55 <= div <= 56:
        return 'I', 'ALOJAMENTO E ALIMENTAÇÃO'
    elif 58 <= div <= 63:
        return 'J', 'INFORMAÇÃO E COMUNICAÇÃO'
    elif 64 <= div <= 66:
        return 'K', 'ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS'
    elif div == 68:
        return 'L', 'ATIVIDADES IMOBILIÁRIAS'
    elif 69 <= div <= 75:
        return 'M', 'ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS'
    elif 77 <= div <= 82:
        return 'N', 'ATIVIDADES ADMINISTRATIVAS E SERVIÇOS COMPLEMENTARES'
    elif div == 84:
        return 'O', 'ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL'
    elif div == 85:
        return 'P', 'EDUCAÇÃO'
    elif 86 <= div <= 88:
        return 'Q', 'SAÚDE HUMANA E SERVIÇOS SOCIAIS'
    elif 90 <= div <= 93:
        return 'R', 'ARTES, CULTURA, ESPORTE E RECREAÇÃO'
    elif 94 <= div <= 96:
        return 'S', 'OUTRAS ATIVIDADES DE SERVIÇOS'
    elif div == 97:
        return 'T', 'SERVIÇOS DOMÉSTICOS'
    elif div == 99:
        return 'U', 'ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕES EXTRATERRITORIAIS'
    else:
        return None, None

# Aplicar o mapeamento
cnae[['sigla_secao', 'desc_secao']] = cnae['divisao'].apply(lambda x: pd.Series(mapear_secao(x)))

# Criar a dimensão final
dim_cnae = cnae[['codigo', 'descricao', 'sigla_secao', 'desc_secao']]


In [16]:
# Caminho para o banco final
conn_gold = sqlite3.connect("../outputs/gold.db")

# Salvar a tabela fato
fato.to_sql("fato_empresas", conn_gold, if_exists="replace", index=False)

# Salvar a dimensão CNAE
dim_cnae.to_sql("dim_cnae", conn_gold, if_exists="replace", index=False)

# Salvar a dimensão Município, se for necessária para join no Power BI
municipio.to_sql("dim_municipio", conn_gold, if_exists="replace", index=False)



5572

In [17]:
# Criar índice na coluna cnpj
with conn_gold:
    conn_gold.execute("CREATE INDEX idx_fato_empresas_cnpj ON fato_empresas(cnpj);")

conn_gold.execute("CREATE INDEX idx_dim_cnae_codigo ON dim_cnae(codigo);")
conn_gold.execute("CREATE INDEX idx_dim_municipio_codigo ON dim_municipio(codigo);")

# Fechar a conexão
conn_gold.close()